In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [3]:
from google.colab import files
uploaded = files.upload()


Saving Nordstrom_products_with_quantity_sold.csv to Nordstrom_products_with_quantity_sold.csv


In [4]:
df = pd.read_csv('Nordstrom_products_with_quantity_sold.csv')


In [5]:
print(df.columns)


Index(['TID', 'DEPARTMENT', 'CATEGORY', 'SUBCATEGORY', 'SORT_BY',
       'RANK_SALES_CATEGORY', 'PRODUCT_URL', 'SKU', 'SKU_VARIANT',
       'PRODUCT_NAME', 'BRAND', 'GENDER', 'PRICE_RETAIL', 'PRICE_CURRENT',
       'RETAIL_PRICE_RAW', 'CURRENT_PRICE_RAW', 'REVIEW_RATING',
       'REVIEW_COUNT', 'PROMOTION', 'AVAILABILITY', 'INVENTORY_WARNING',
       'COLOR', 'COLOR_CODE', 'RunID', 'RunDate', 'InsertUpdateTime',
       'QUANTITY_SOLD'],
      dtype='object')


In [6]:
def calculate_price_elasticity(row):
    # Avoid division by zero by checking for quantity sold being 0
    if row['QUANTITY_SOLD'] == 0:
        return 0

    # Simple elasticity calculation (adjust as needed)
    price = row['PRICE_CURRENT']
    quantity = row['QUANTITY_SOLD']
    elasticity = -0.1 * (price / quantity)  # Adjust this multiplier if necessary

    return elasticity

In [7]:
# Apply the elasticity calculation to each row
df['PRICE_ELASTICITY'] = df.apply(calculate_price_elasticity, axis=1)


In [8]:
def adjust_price(row, elasticity):
    current_price = row['PRICE_CURRENT']

    # Calculate the adjusted price based on elasticity
    adjusted_price = current_price * (1 + 0.1 * elasticity)  # Adjust price based on elasticity

    # Enforce a minimum price change of 5% (positive adjustment)
    min_price_change = 0.05  # Minimum 5% change
    if abs(adjusted_price - current_price) < current_price * min_price_change:
        adjusted_price = current_price * (1 + min_price_change)  # Enforce at least 5% increase

    # Ensure adjusted price is not less than the current price (no losses)
    if adjusted_price < current_price:
        adjusted_price = current_price  # Ensure no decrease in price

    return adjusted_price

In [9]:
df['Optimal Price'] = df.apply(lambda row: adjust_price(row, row['PRICE_ELASTICITY']), axis=1)


In [10]:
# Calculate the revenue with adjusted prices
df['Adjusted Revenue'] = df['Optimal Price'] * df['QUANTITY_SOLD']


In [11]:
total_current_revenue = df['PRICE_CURRENT'].sum()
total_adjusted_revenue = df['Adjusted Revenue'].sum()


In [12]:
revenue_difference = total_adjusted_revenue - total_current_revenue

print(f"Total Current Revenue: ${total_current_revenue:,.2f}")
print(f"Total Adjusted Revenue: ${total_adjusted_revenue:,.2f}")
print(f"Revenue Difference: ${revenue_difference:,.2f}")

Total Current Revenue: $90,143.47
Total Adjusted Revenue: $2,062,997.48
Revenue Difference: $1,972,854.01


In [13]:
# Assuming the DataFrame contains 'Brand', 'Product', 'PRICE_CURRENT', 'Adjusted Price'

# Display the first 10 rows with the required information (no 'Cost' column)
df_display = df[['TID','BRAND', 'PRODUCT_NAME', 'PRICE_CURRENT', 'Optimal Price']].head(10)

# Calculate the profit as the difference between the current price and the adjusted price
df_display['Profit'] = df_display['Optimal Price'] - df_display['PRICE_CURRENT']

# Display the final result for the first 10 entries
print(df_display)


         TID                  BRAND  \
0  142453945                  vuori   
1  142453946                     AG   
2  142453947                     AG   
3  142453948                   Veja   
4  142453949  Billionaire Boys Club   
5  142453950                  vuori   
6  142453951               Rainbow®   
7  142453952              Nordstrom   
8  142453953            Rodd & Gunn   
9  142453954           TravisMathew   

                                 PRODUCT_NAME  PRICE_CURRENT  Optimal Price  \
0       Strato Slim Fit Crewneck Tech T-Shirt          54.00        56.7000   
1             Everett Slim Straight Leg Jeans         132.66       139.2930   
2  Men's Graduate Tailored Straight Leg Jeans         138.60       145.5300   
3                               Campo Sneaker         175.00       175.0000   
4                 Creation Cotton Graphic Tee          48.00        50.4000   
5                            Strato Tech Polo          64.00        67.2000   
6                